In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
finance_news = pd.read_excel('finance_data/SmoothNLP专栏资讯数据集样本10k.xlsx')
finance_news = finance_news.drop(columns = ['Unnamed: 0'])

In [19]:
finance_news = finance_news[~finance_news['content'].isna()]

In [4]:
finance_news.columns

Index(['title', 'content', 'pub_ts'], dtype='object')

In [5]:
finance_news['title'].iloc[0]

'引入前美团地推大将，每日优鲜便利购如何上演“攻守道”?'

In [6]:
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize

In [11]:
summarize(finance_news['content'].iloc[2])

'这就形成了认知矛盾：当越来越多的人习惯性的将理财资金、银行卡等深度绑定到支付宝和微信之后，用户想要的扫码取款，是微信和支付宝扫码交易式的一键方便快捷操作。而近年银行推出的扫码取款，大多都是流于“扫码”形式，其本质仍然没有脱离于传统银行的流程架构，其体验也不如互联网产品的高效方便。\n根据兰州银行的宣传材料，微信和支付宝用户在兰州银行任一ATM机具上点击“二维码取款”，输入取款金额后，ATM机具屏幕自动生成取款二维码，这时用户打开微信或者支付宝扫描二维码，输入支付密码，ATM机器就会自动吐钞。\n正当大家思考是不是以后各银行都可以使用支付宝和微信在ATM机上扫码取款的时候，兰州银行的这个行为被紧急叫停了。而且是银行扫码取款问世一天后就被叫停了。\n9月5日，兰州银行官方微信中关于“扫码取款”的消息也已被删除，当晚被暂停。9月6日，总行层面发文明确要求叫停。\n“微信、支付宝等支付账户不能提取现金，支付企业也不能经营货币。经营货币是商业银行等金融机构才有的职能，支付账户与银行账户是两个不同的体系。”\n董希淼认为，就具体业务而言，兰州银行此举意义不大。“支付宝、微信本身就是捆绑银行卡，如果要取现，直接用银行卡不是更直接?如果不想用银行卡，无卡取现、刷脸取现也已经日益普遍了。”\n除了上述的监管问题，其中利益分配当然也是重要原因。毕竟在传统银行业，互联网想要以便利性打破传统壁垒也要付出不小的代价，此前支付宝们与银行不断的限额、手续费摩擦也证明了这点。\n所以，像微信支付宝转账交易式方便快捷的“扫码取现”其实离我们已经不远，应该会在2年到5年间实现普及，毕竟在整体现金交易逐年递减、线上交易成为大趋势的现在，扫码取现这一需求只是很小的需求，它最终会随着互联网的大潮而实现，最终成为在“无现金”环境下装点银行业门面的小亮点罢了。'

In [12]:
import jieba
jieba.enable_paddle()

Paddle enabled successfully......


In [13]:
tokens = jieba.cut("所以，整个过程有点像勘探。首先是有人发起，然后有千百人买了镐和铁锹跟进。有人会满载而归，但看走眼的人也会空手回来。"
                   ,cut_all=False)

In [14]:
from zhon.hanzi import punctuation as punctuation_zh
from string import punctuation
from words_utils import tokenization as finance_token
import re

In [15]:
def clean_text(content):
    content = re.sub('\xa0 ?|\u3000+', '', content)
    content = re.sub(' ?\n+', '\n', content)
    content = content.strip('\n')
    return content

In [20]:
clean_content = finance_news['content'].apply(clean_text)

In [22]:
clean_content_length = clean_content.apply(lambda x: len(x))

In [26]:
clean_content_length.describe()

count     9994.000000
mean      2743.330698
std       1176.500166
min          0.000000
25%       1959.000000
50%       2620.500000
75%       3376.000000
max      18431.000000
Name: content, dtype: float64

In [25]:
len(clean_content[0])

2330

In [28]:
from albertlib.tokenization import BasicTokenizer

In [29]:
tokenizer = BasicTokenizer()

In [31]:
len(tokenizer.tokenize(clean_content[0]))

2251

In [31]:
stop_words = finance_token.read_stop_words()
tokenizer = finance_token.JiebaTokenizer(stop_words, [punctuation, punctuation_zh], vocab = 'finance_data/vocab')

In [57]:
finance_news['title'].iloc[3]

'王石遭“疯抢”！为什么那么多公司想聘请他当董事长？'

In [58]:
finance_news['content'].iloc[3]

'\n\xa0 \xa0 \xa0 67岁的王石，终究还是回到了他熟悉的商业领域这一战场。在此前一段时间，王石似乎是习惯性地过起了“佛系”生活。在之前的一年时间中，王石身上的标签是“深潜”、“划艇”、“公益”、“艺术”、“生态”。王石做网红大叔好像“上瘾”了，褪去那种在商界叱咤风云的霸气。\n\xa0 \xa0 \xa0 但王石终究不是一个会耐得住寂寞的人，今年成为了两家公司的联席董事长。值得一提的是，华大集团、远大集团的董事长都是王石的好朋友，因此王石才答应成为联席董事长。至于其他邀请自己的公司，王石通通都拒绝了。不过面对这样的“疯抢”，事实上也证明着王石有着自身独特的魅力。\n\xa0 \xa0 \xa0 有句老话说得好，“人的名，树的影”。执掌万科多年的王石，早就成为商界的标志性人物。再说了，王石还是8848手机的代言人！唔，也算是手机圈的“另类网红”了。而且能够在万科这样一个巨型平台上游刃有余地奋斗那么多年，自然能够反映出王石有着足够强的能力。\n\n\xa0 \xa0 \xa0 当然，更被其他公司注重的是王石的人脉和资源等。在商界有着极大影响力的王石，哪怕不负责公司的任何具体事务，只要挂个名就能让这家公司瞬间成为关注的焦点。\n\xa0 \xa0 \xa0 比如就在王石宣布成为华大集团联席董事长后，华大集团的股价当天就一路飙涨，甚至实现连续数天的狂涨势头。如果王石动用自己的人脉和资源，更有可能为公司插上腾飞的翅膀。看来，王石本身的影响力和“朋友圈”是他受到“疯抢”的根本原因。\n\n\xa0 \xa0 \xa0 但必须要指出的是，王石最熟悉的还是地产圈。而华大集团和远大集团的主要业务是有关生物科技、环保方面，王石究竟能够发挥出多少能力还有待商榷。比如华大集团表示王石将发挥其在经营管理、制度建设、商业运作等方面的丰富经验和卓越能力，协助董事长管理集团事宜。不过王石真能像管理地产集团那样，去管理生物科技公司吗？我们还不得而知。\n\xa0 \xa0 \xa0 而就目前来看，王石在这些公司似乎还偏向于“玄学”意味的管理模式。就像王石提到的那样，“无论是代表传统的还是代表未来的企业，还有我的位置，很多基本的逻辑和道理是一样的”。这是不是意味着王石并没有真正的全面突破，还是会延续既有的管理经验呢？结果到底如何，一切都交给时间去验证吧！（科技新发现 康斯坦丁/文）\n

In [5]:
all_token_length = []
with open('finance_data/data.txt','r', encoding='UTF-8') as file:
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        tokens = tokenizer.tokenize(line)
        all_token_length.append(len(tokens))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.584 seconds.
Prefix dict has been built successfully.


In [11]:
all_token_length = pd.DataFrame(all_token_length)

In [13]:
all_token_length.describe()

,0
count,296933.000000
mean,25.683282
std,22.148801
min,0.000000
25%,10.000000
50%,23.000000
75%,37.000000
max,1902.000000


In [38]:
all_token_length.quantile(q = 0.80)

0    40.0
Name: 0.8, dtype: float64